In [9]:
import cv2 as cv
import numpy as np
import time
from skimage.feature import local_binary_pattern
from scipy.stats import mode

In [10]:
image_train=np.load("image_train.npy")
label_train=np.load("label_train.npy")
image_test=np.load("image_test.npy")
label_test=np.load("label_test.npy")

In [11]:
radius = 1
n_points = 8 * radius
method = 'uniform'

LBP_train = [
    local_binary_pattern(cv.cvtColor(img, cv.COLOR_RGB2GRAY), n_points, radius, method)
    for img in image_train
]
LBP_test = [
    local_binary_pattern(cv.cvtColor(img, cv.COLOR_RGB2GRAY), n_points, radius, method)
    for img in image_test
]

In [12]:
n_bins = n_points + 2
LBP_train_histogram = np.array([
    cv.calcHist([lbp.astype('float32')], [0], None, [n_bins], [0, n_bins]).flatten()
    for lbp in LBP_train
])
LBP_test_histogram = np.array([
    cv.calcHist([lbp.astype('float32')], [0], None, [n_bins], [0, n_bins]).flatten()
    for lbp in LBP_test
])
'''
LBP_all=np.concatenate((LBP_train_histogram,LBP_test_histogram),axis=0)
mean = np.mean(LBP_all, axis=0)
std = np.std(LBP_all, axis=0)

LBP_train_histogram = (LBP_train_histogram - mean) / std
LBP_test_histogram = (LBP_test_histogram - mean) / std

'''
y_train = label_train.flatten()
y_test = label_test.flatten()

In [16]:
def knn_predict(X_train, y_train, X_test, k=3):
    predictions = []
    for x_test in X_test:
        # Compute distances to all training points
        distances = np.linalg.norm(X_train - x_test, axis=1)

        # Find the k nearest neighbors
        k_indices = np.argsort(distances)[:k]
        k_labels = y_train[k_indices]

        # Determine the most common label
        unique_labels, counts = np.unique(k_labels, return_counts=True)
        most_common = unique_labels[np.argmax(counts)]
        
        predictions.append(most_common)

    return np.array(predictions)


k = 1  # Number of neighbors
start_time = time.time()

y_pred = knn_predict(LBP_train_histogram, y_train, LBP_test_histogram[:100], k=k)  # Use a subset of test data for speed

accuracy = np.mean(y_pred == y_test[:100])  # Match the subset size
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 96.00%
